## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2023xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [55]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [56]:
from datetime import datetime
import json
from tab_dataset.cdataset import Cdataset
import pandas as pd
import ntv_pandas

def analyse_integrite(data, fields, affiche=True, indic=True):
    '''analyse les relations du DataFrame 'data' définies dans le dict 'fields'.
    Le nombre de ligne en erreur par relation (dict) est retourné et optionnellement ('indic') affiché. 
    La liste des lignes en erreur est optionnellement ajoutée ('affiche') à 'data' sous forme de champs booléens par relation.
    '''
    dic_errors = Cdataset(data).check_relationship(fields)
    dic_count = {name: len(errors) for name, errors in dic_errors.items()}
    if affiche:
        for name, total in dic_count.items():
            print('{:<50} {:>5}'.format(name, total))
    if indic:
        data['ok'] = True
        for name, errors in dic_errors.items():
            data[name] = True
            data.loc[errors, name] = False
            data['ok'] = data['ok'] & data[name] 
    return dic_count

def add_nbre_pdc(data, affiche=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    if affiche:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [57]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31', '2023-11-23']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
#    'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/gen-ntv/python/Validation/irve/Analyse/'}
#log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
    'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
    #'chemin': 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
      }
irve = pd.read_csv(log['chemin'] + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  54296


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [58]:
# complément à inclure dans le schéma de données
fields = [
 # relation unicité des pdl
     { "name": "index",
       "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 0,5 % des points de charge sont hors itinerance

In [59]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'])
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  2535
nombre de pdc en itinerance   :  51761


In [60]:
# arborescence des champs et nombre de valeurs différentes
print(Cdataset(itinerance_init).tree())

-1: root-derived (51761)
   0 : index (0 - 51761)
   1 : contact_operateur (51632 - 129)
   2 : nom_enseigne (50981 - 780)
   3 : coordonneesXY (36589 - 15172)
   4 : adresse_station (36559 - 15202)
   5 : id_station_itineranc (35192 - 16569)
   6 : nom_station (36110 - 15651)
      9 : condition_acces (15649 - 2)
   7 : implantation_station (51756 - 5)
   8 : nbre_pdc (51708 - 53)
   10: horaires (51263 - 498)
   11: station_deux_roues (51755 - 6)
   12: id_pdc_itinerance (524 - 51237)
   13: date_maj (51079 - 682)
   14: last_modified (51313 - 448)


-----------------------
## 4 - Bilan initial intégrité
- 10 % des lignes présentent un défaut d'intégrité

In [61]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                           1046
contact_operateur - id_station_itinerance            608
nom_enseigne - id_station_itinerance                 875
coordonneesXY - id_station_itinerance               1426
id_station_itinerance - id_pdc_itinerance             76
nom_station - id_station_itinerance                  906
implantation_station - id_station_itinerance         296
nbre_pdc - id_station_itinerance                     942
condition_acces - id_station_itinerance                9
horaires - id_station_itinerance                      33
station_deux_roues - id_station_itinerance           488
adresse_station - coordonneesXY                     1973

nombre d'enregistrements sans erreurs :  46633
nombre d'enregistrements avec au moins une erreur :  5128
taux d'erreur :  10  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 10% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 10% le nombre de lignes erronnées

In [62]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'], ' soit ', round(log['sans_doublons_pdc']/log['init_ko'] * 100), ' %')

nombre de doublons pdc :  524  soit  10  %
nombre de pdc sans doublon   :  4604  soit  90  %


In [63]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             53
nom_enseigne - id_station_itinerance                 314
coordonneesXY - id_station_itinerance               1007
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  639
implantation_station - id_station_itinerance         253
nbre_pdc - id_station_itinerance                     935
condition_acces - id_station_itinerance                5
horaires - id_station_itinerance                      29
station_deux_roues - id_station_itinerance           479
adresse_station - coordonneesXY                     1963

nombre d'enregistrements sans erreurs :  484
nombre d'enregistrements avec au moins une erreur :  4120


-----------------------
## 6 - Séparation doublons station - date de maj
- la suppression des anciens pdc permet de réduire de 10% supplémentaire le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 35 opérateurs et sont liées à des causes multiples

In [64]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  533  soit  10  %
nombre de pdc sans doublon   :  3587


In [65]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                 263
coordonneesXY - id_station_itinerance                433
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  367
implantation_station - id_station_itinerance          21
nbre_pdc - id_station_itinerance                     670
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                      25
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                     1916

nombre d'enregistrements sans erreurs :  102
nombre d'enregistrements avec au moins une erreur :  3485  soit  85  %


In [66]:
# structure des données présentant une erreur et nombre de valeurs
print(Cdataset(itinerance_5).tree())

-1: root-derived (3485)
   0 : index (0 - 3485)
   1 : contact_operateur (3450 - 35)
   2 : nom_enseigne (3153 - 332)
      9 : condition_acces (330 - 2)
   3 : coordonneesXY (2641 - 844)
   4 : adresse_station (2236 - 1249)
   5 : id_station_itineranc (1918 - 1567)
      13: date_maj (1481 - 86)
   6 : nom_station (2261 - 1224)
      14: last_modified (1165 - 59)
   7 : implantation_station (3480 - 5)
   8 : nbre_pdc (3466 - 19)
   10: horaires (3444 - 41)
   11: station_deux_roues (3480 - 5)
   12: id_pdc_itinerance (0 - 3485)


In [67]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  11


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [68]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  3485
total des doublons à supprimer :  1057
nombre de pdc avec controles ok :  47219


In [69]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [70]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, affiche=False)
log['bilan_erreurs'] = sum(res.values())
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [71]:
# structure des données bonnes
print(Cdataset(itinerance_ok.loc[:, mandatory]).tree())

-1: root-derived (47219)
   4 : id_station_itineranc (32245 - 14974)
      1 : nom_enseigne (14512 - 462)
      2 : coordonneesXY (894 - 14080)
         0 : contact_operateur (13964 - 116)
         3 : adresse_station (280 - 13800)
         8 : condition_acces (14078 - 2)
      5 : nom_station (708 - 14266)
      6 : implantation_station (14969 - 5)
      7 : nbre_pdc (14921 - 53)
      9 : horaires (14508 - 466)
      10: station_deux_roues (14968 - 6)
      19: paiement_autre (14967 - 7)
      24: last_modified (14594 - 380)
   11: id_pdc_itinerance (0 - 47219)
   12: puissance_nominale (47121 - 98)
   13: prise_type_ef (47211 - 8)
   14: prise_type_2 (47211 - 8)
   15: prise_type_combo_ccs (47212 - 7)
   16: prise_type_chademo (47212 - 7)
   17: prise_type_autre (47213 - 6)
   18: paiement_acte (47211 - 8)
   20: reservation (47212 - 7)
   21: accessibilite_pmr (47215 - 4)
   22: restriction_gabarit (47116 - 103)
   23: date_maj (46558 - 661)


### Indicateurs

In [72]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [73]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

5  opérateurs représentent :  78  % des écarts

support@alizecharge.fr 858
sav@izivia.com 723
supervision-ev.france@totalenergies.com 554
assistance-commerciale@e-vadea.fr 321
pascal.lhermitte@siege27.fr 260


In [74]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, affiche=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           1704
49  %


In [75]:
# stockage des indicateurs
log |= indic
with open('logfile.txt', 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (4) (3 stations)
- exemple station 'FRS22P22220001': 2 coordonnées XY, 2 adresses, 2 nom_station, 2 implantation station

In [76]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
71,354,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",AVENUE DE PARIS VERSAILLES,FR55CP78000,AVENUE DE PARIS VERSAILLES,Voirie,4,Accès réservé,24/7,FALSE,FR55CE780004880188021299331,2022-11-03,2023-11-23T06:33:56.437000+00:00
72,355,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",AVENUE DE PARIS VERSAILLES,FR55CP78000,AVENUE DE PARIS VERSAILLES,Voirie,4,Accès réservé,24/7,FALSE,FR55CE780004880188021299351,2022-11-03,2023-11-23T06:33:56.437000+00:00
160,620,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",6 AVENUE DE PARIS -VERSAILLES,FR55CP78000,6 AVENUE DE PARIS -VERSAILLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CEFR78000AVDEPAR1S0,2022-11-03,2023-11-23T06:33:56.437000+00:00
161,621,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",6 AVENUE DE PARIS -VERSAILLES,FR55CP78000,6 AVENUE DE PARIS -VERSAILLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CEFR78000AVDEPAR1S1,2022-11-03,2023-11-23T06:33:56.437000+00:00
1403,35040,support@alizecharge.fr,Ouest Charge,"[-3.252257, 48.239723]",2 Rue de la Gare 22110 PLOUGUERNEVEL,FRS22P22220001,Plouguernevel-Le bourg - route de Rostrenen,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200011,2023-07-26,2023-11-23T06:35:56.448000+00:00
1404,35041,support@alizecharge.fr,Ouest Charge,"[-3.252257, 48.239723]",2 Rue de la Gare 22110 PLOUGUERNEVEL,FRS22P22220001,Plouguernevel-Le bourg - route de Rostrenen,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200012,2023-07-26,2023-11-23T06:35:56.448000+00:00
3086,50179,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550401,2023-07-26,2023-11-23T06:35:56.448000+00:00
3087,50180,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550402,2023-07-26,2023-11-23T06:35:56.448000+00:00
3088,50181,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550403,2023-07-26,2023-11-23T06:35:56.448000+00:00
3089,50182,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550404,2023-07-26,2023-11-23T06:35:56.448000+00:00


In [77]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FRS22P22220001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1403,35040,support@alizecharge.fr,Ouest Charge,"[-3.252257, 48.239723]",2 Rue de la Gare 22110 PLOUGUERNEVEL,FRS22P22220001,Plouguernevel-Le bourg - route de Rostrenen,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200011,2023-07-26,2023-11-23T06:35:56.448000+00:00
1404,35041,support@alizecharge.fr,Ouest Charge,"[-3.252257, 48.239723]",2 Rue de la Gare 22110 PLOUGUERNEVEL,FRS22P22220001,Plouguernevel-Le bourg - route de Rostrenen,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200012,2023-07-26,2023-11-23T06:35:56.448000+00:00
1405,35042,support@alizecharge.fr,Ouest Charge,"[-3.313195, 48.242261]",0 ZA de Goasnel 22110 ROSTRENEN,FRS22P22220001,Rostrenen-ZA de Goasnel,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200013,2023-07-26,2023-11-23T06:35:56.448000+00:00
1406,35043,support@alizecharge.fr,Ouest Charge,"[-3.313195, 48.242261]",0 ZA de Goasnel 22110 ROSTRENEN,FRS22P22220001,Rostrenen-ZA de Goasnel,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200014,2023-07-26,2023-11-23T06:35:56.448000+00:00


### Repartition des defauts par operateur
Répartition pour les 5 opérateurs présentant le plus de défauts

In [121]:
print('PdC avec une incohérence : ')
for operateur in oper:
    print(sum(itinerance_4.contact_operateur == operateur))
    print('\nRépartition des incohérences ', operateur, ' :')
    for cont in controles:
        #print(cont, sum((itinerance_4['contact_operateur'] == 'supervision-ev.france@totalenergies.com') & ~(itinerance_4[cont])))
        print('{:<50} {:>5}'.format(cont, sum((itinerance_4['contact_operateur'] == operateur) & ~(itinerance_4[cont]))))

PdC avec une incohérence : 
858

Répartition des incohérences  support@alizecharge.fr  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                   0
coordonneesXY - id_station_itinerance                 43
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  129
implantation_station - id_station_itinerance          17
nbre_pdc - id_station_itinerance                     117
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                       0
station_deux_roues - id_station_itinerance             0
adresse_station - coordonneesXY                      817
723

Répartition des incohérences  sav@izivia.com  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                   0
coo

### Cohérence implantation_station - id_station
- 21 pdc sont liés à une erreur de choix d'implantation (couplé à d'autres erreurs)

In [78]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
42,308,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300611,2022-11-03,2023-11-23T06:33:56.437000+00:00
43,309,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Voirie,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300621,2022-11-03,2023-11-23T06:33:56.437000+00:00
44,310,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300631,2022-11-03,2023-11-23T06:33:56.437000+00:00
45,311,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300641,2022-11-03,2023-11-23T06:33:56.437000+00:00
384,4251,support@alizecharge.fr,Métropole Rouen Normandie,"[0.872578, 49.479934]",Avenue du Président Coty 76480 DUCLAIR,FRC01P76222001,DUCLAIR - Avenue du Président Coty,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRC01E762220011,2023-07-26,2023-11-23T06:35:56.448000+00:00
385,4252,support@alizecharge.fr,Métropole Rouen Normandie,"[0.872578, 49.479934]",Avenue du Président Coty 76480 DUCLAIR,FRC01P76222001,DUCLAIR - Avenue du Président Coty,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRC01E762220012,2023-07-26,2023-11-23T06:35:56.448000+00:00
386,4253,support@alizecharge.fr,Métropole Rouen Normandie,"[0.87699, 49.48461]",52 Chemin du Catel 76480 DUCLAIR,FRC01P76222001,DUCLAIR - Parking Chemin du Catel,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRC01E762220013,2023-07-26,2023-11-23T06:35:56.448000+00:00
387,4254,support@alizecharge.fr,Métropole Rouen Normandie,"[0.87699, 49.48461]",52 Chemin du Catel 76480 DUCLAIR,FRC01P76222001,DUCLAIR - Parking Chemin du Catel,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRC01E762220014,2023-07-26,2023-11-23T06:35:56.448000+00:00
1403,35040,support@alizecharge.fr,Ouest Charge,"[-3.252257, 48.239723]",2 Rue de la Gare 22110 PLOUGUERNEVEL,FRS22P22220001,Plouguernevel-Le bourg - route de Rostrenen,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200011,2023-07-26,2023-11-23T06:35:56.448000+00:00
1404,35041,support@alizecharge.fr,Ouest Charge,"[-3.252257, 48.239723]",2 Rue de la Gare 22110 PLOUGUERNEVEL,FRS22P22220001,Plouguernevel-Le bourg - route de Rostrenen,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200012,2023-07-26,2023-11-23T06:35:56.448000+00:00


### Cohérence nom_station - id_station
- 367 pdc sont associés à une station avec un nom non cohérent

In [79]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
47,316,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643461,2022-11-03,2023-11-23T06:33:56.437000+00:00
48,317,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643471,2022-11-03,2023-11-23T06:33:56.437000+00:00
49,318,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684251,2022-11-03,2023-11-23T06:33:56.437000+00:00
50,319,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684261,2022-11-03,2023-11-23T06:33:56.437000+00:00
51,320,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.768425, 43.958425]",PARKING ELEPHANT BLEU - LES ANGLES,FR55CP30133,PARKING ELEPHANT BLEU - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334396228947525161,2022-11-03,2023-11-23T06:33:56.437000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3086,50179,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550401,2023-07-26,2023-11-23T06:35:56.448000+00:00
3087,50180,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550402,2023-07-26,2023-11-23T06:35:56.448000+00:00
3088,50181,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550403,2023-07-26,2023-11-23T06:35:56.448000+00:00
3089,50182,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550404,2023-07-26,2023-11-23T06:35:56.448000+00:00


### Coherence adresse - coordonnees
- 1916 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [80]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
18,119,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","400 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:00,false,FR3R3E10000849681,2023-11-22,2023-11-23T06:36:07.508000+00:00
19,120,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","401 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:01,false,FR3R3E10000849682,2023-11-22,2023-11-23T06:36:07.508000+00:00
20,121,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",708 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849691,2023-11-22,2023-11-23T06:36:07.508000+00:00
21,122,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",709 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849692,2023-11-22,2023-11-23T06:36:07.508000+00:00
22,137,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",713 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849851,2023-11-22,2023-11-23T06:36:07.508000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3582,54117,advenir@zeborne.com,GARAGE NELLO CHELLI - DIJON,"[5.065799, 47.354481]",23 RUE DES ARDENNES 21000 Dijon,FRZTLE22AC55087,Toyota - Dijon - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE22AC55087,2021-02-19,2023-11-23T06:37:11.596000+00:00
3583,54126,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2023-11-23T06:37:10.574000+00:00
3584,54127,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2023-11-23T06:37:10.574000+00:00
3585,54128,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2023-11-23T06:37:10.574000+00:00


- exemple : une station aux bonnes coordonnées avec plusieurs adresses -> erreur de saisie ?

In [81]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[2.87930851314442, 48.94679007929618]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2918,49691,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",50 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003954,2023-01-27,2023-11-23T06:34:34.653000+00:00
2919,49692,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",51 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003955,2023-01-27,2023-11-23T06:34:34.653000+00:00
2920,49693,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003958,2023-01-27,2023-11-23T06:34:34.653000+00:00
2921,49694,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003959,2023-01-27,2023-11-23T06:34:34.653000+00:00


- exemple : Stations Ouest charge (double saisie par deux opérateurs)

In [82]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-3.577855, 48.771084]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1402,35137,support@alizecharge.fr,Ouest Charge,"[-3.577855, 48.771084]",0 parking place du peillet 22560 TREBEURDEN,FRS22P22343001,Trebeurden-Place du Peillet,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E223430011,2023-07-26,2023-11-23T06:35:56.448000+00:00
1403,35138,support@alizecharge.fr,Ouest Charge,"[-3.577855, 48.771084]",0 parking place du peillet 22560 TREBEURDEN,FRS22P22343001,Trebeurden-Place du Peillet,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E223430012,2023-07-26,2023-11-23T06:35:56.448000+00:00
1586,35341,sav@izivia.com,OUEST CHARGE - 22,"[-3.577855, 48.771084]",PARKING PLACE DU PEILLET,FRS22PSDE22311,TREBEURDEN-PLACE DU PEILLET,Voirie,2,Accès libre,24/7,false,FRS22ESDE22311,2023-04-07,2023-11-23T06:32:19.689000+00:00
1587,35342,sav@izivia.com,OUEST CHARGE - 22,"[-3.577855, 48.771084]",PARKING PLACE DU PEILLET,FRS22PSDE22312,TREBEURDEN-PLACE DU PEILLET,Voirie,2,Accès libre,24/7,false,FRS22ESDE22312,2023-04-07,2023-11-23T06:32:19.689000+00:00


### Cohérence station - coordonnées
- 477 pdc sont associés à des stations avec plusieurs coordonnées

In [83]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
420,6928,contact@reseau-eborn.fr,eborn,"[6.323481, 45.883111]",Rue du Capricorne 74230 Thônes,FREBNPYQIQWQ8UZY,"Thônes, Parking Clos",Voirie,4,Accès libre,24/7,false,FREBNESZAZ2,2023-11-07,2023-11-23T06:33:49.428000+00:00
759,9863,web@freshmile.com,Freshmile,"[2.378329, 48.681627]",21 Avenue De Joinville 91170 Viry-Châtillon,FRFR1PSBPX6OZUXS,"Viry-Châtillon, avenue du Général de Gaulle",Voirie,8,Accès libre,24/7,false,FRFR1EAAAD1,2023-11-22,2023-11-23T06:37:30.166000+00:00
761,10454,web@freshmile.com,Freshmile,"[2.378329, 48.681627]",21 Avenue De Joinville 91170 Viry-Châtillon,FRFR1PSBPX6OZUXS,"Viry-Châtillon, avenue du Général de Gaulle",Voirie,8,Accès libre,24/7,false,FRFR1EBUTF1,2023-11-22,2023-11-23T06:37:30.166000+00:00
767,11389,web@freshmile.com,Freshmile,"[2.378329, 48.681627]",21 Avenue De Joinville 91170 Viry-Châtillon,FRFR1PSBPX6OZUXS,"Viry-Châtillon, avenue du Général de Gaulle",Voirie,8,Accès libre,24/7,false,FRFR1EFUWM1,2023-11-22,2023-11-23T06:37:30.166000+00:00
770,11742,web@freshmile.com,Freshmile,"[3.135937, 45.765259]",15 Rue Jean Claret 63000 Clermont-Ferrand,FRFR1PGJTASUUHDC,"Clermont-Ferrand, Openium",Voirie,4,Accès libre,24/7,false,FRFR1EGVWU1,2023-11-22,2023-11-23T06:37:30.166000+00:00
772,11914,web@freshmile.com,Freshmile,"[3.1361313, 45.76525364]",15 Rue Jean Claret 63000 Clermont-Ferrand,FRFR1PGJTASUUHDC,"Clermont-Ferrand, Openium",Voirie,4,Accès libre,24/7,false,FRFR1EHHXV1,2023-11-22,2023-11-23T06:37:30.166000+00:00
774,12288,web@freshmile.com,Freshmile,"[1.391725, 43.566937]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1EJVVE1,2022-11-09,2023-11-23T06:37:30.166000+00:00
775,12289,web@freshmile.com,Freshmile,"[0.66337, 43.85402]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1EJVVE2,2022-11-09,2023-11-23T06:37:30.166000+00:00
776,12497,web@freshmile.com,Freshmile,"[6.082335, 43.243236]",87 Chemin de la Pépinière 13600 La Ciotat,FRFR1PD0AIJIWGHD,"La Ciotat, 87 Chemin de la Pépinière - 3553",Voirie,2,Accès libre,24/7,false,FRFR1EKLMQ1,2022-11-09,2023-11-23T06:37:30.166000+00:00
777,12498,web@freshmile.com,Freshmile,"[6.177948, 44.069971]",87 Chemin de la Pépinière 13600 La Ciotat,FRFR1PD0AIJIWGHD,"La Ciotat, 87 Chemin de la Pépinière - 3553",Voirie,2,Accès libre,24/7,false,FRFR1EKLMQ2,2022-11-09,2023-11-23T06:37:30.166000+00:00


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [84]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,6,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2023-11-23T06:31:56.615000+00:00
1,7,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2023-11-23T06:31:56.615000+00:00
2,8,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2023-11-23T06:31:56.615000+00:00
3,9,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2023-11-23T06:31:56.615000+00:00
5,11,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2023-11-23T06:31:56.615000+00:00
6,12,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2023-11-23T06:31:56.615000+00:00
7,13,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2023-11-23T06:31:56.615000+00:00
8,14,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2023-11-23T06:31:56.615000+00:00
9,15,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2023-11-23T06:31:56.615000+00:00
10,16,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2023-11-23T06:31:56.615000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [85]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
52,327,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6A61,2022-11-03,2023-11-23T06:33:56.437000+00:00
53,328,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B62,2022-11-03,2023-11-23T06:33:56.437000+00:00
54,329,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B63,2022-11-03,2023-11-23T06:33:56.437000+00:00
55,330,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2023-11-23T06:33:56.437000+00:00
56,331,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A2,2022-11-03,2023-11-23T06:33:56.437000+00:00
57,332,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A3,2022-11-03,2023-11-23T06:33:56.437000+00:00


- exemple : plusieurs stations d'opérateurs différents avec les mêmes coordonnées -> suppression non effectuée lors d'un changement d'opérateur ?

In [86]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.658987, 48.542091]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2322,36122,support@alizecharge.fr,Ouest Charge,"[-4.658987, 48.542091]",Rue Auguste Caroff 29830 PLOUDALMEZEAU,FRS29P29178001,Ploudalmezeau-Rue Auguste Caroff,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS29E291780011,2023-07-26,2023-11-23T06:35:56.448000+00:00
2323,36123,support@alizecharge.fr,Ouest Charge,"[-4.658987, 48.542091]",Rue Auguste Caroff 29830 PLOUDALMEZEAU,FRS29P29178001,Ploudalmezeau-Rue Auguste Caroff,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS29E291780012,2023-07-26,2023-11-23T06:35:56.448000+00:00
2548,36356,sav@izivia.com,OUEST CHARGE - 29,"[-4.658987, 48.542091]",RUE AUGUSTE CAROFF,FRS29PSD2912011,PLOUDALMEZEAU-RUE AUGUSTE CAROFF,Voirie,2,Accès libre,24/7,false,FRS29ESD2912011,2023-04-14,2023-11-23T06:32:20.025000+00:00
2549,36357,sav@izivia.com,OUEST CHARGE - 29,"[-4.658987, 48.542091]",RUE AUGUSTE CAROFF,FRS29PSD2912012,PLOUDALMEZEAU-RUE AUGUSTE CAROFF,Voirie,2,Accès libre,24/7,false,FRS29ESD2912012,2023-04-14,2023-11-23T06:32:20.025000+00:00


In [87]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.374469, 48.471878]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2216,36012,support@alizecharge.fr,Ouest Charge,"[-4.374469, 48.471878]",Route de Plabennec 29860 KERSAINT PLABENNEC,FRS29P29095001,Kersaint Plabennec-Route de Plabennec,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS29E290950011,2023-07-26,2023-11-23T06:35:56.448000+00:00
2217,36013,support@alizecharge.fr,Ouest Charge,"[-4.374469, 48.471878]",Route de Plabennec 29860 KERSAINT PLABENNEC,FRS29P29095001,Kersaint Plabennec-Route de Plabennec,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS29E290950012,2023-07-26,2023-11-23T06:35:56.448000+00:00
2842,36652,sav@izivia.com,OUEST CHARGE - 29,"[-4.374469, 48.471878]",ROUTE DE PLABENNEC,FRS29PSD296211,KERSAINT PLABENNEC-ROUTE DE PLABENNEC,Voirie,2,Accès libre,24/7,false,FRS29ESD296211,2023-04-14,2023-11-23T06:32:20.025000+00:00
2843,36653,sav@izivia.com,OUEST CHARGE - 29,"[-4.374469, 48.471878]",ROUTE DE PLABENNEC,FRS29PSD296212,KERSAINT PLABENNEC-ROUTE DE PLABENNEC,Voirie,2,Accès libre,24/7,false,FRS29ESD296212,2023-04-14,2023-11-23T06:32:20.025000+00:00


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [88]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Incoherence nbre_pdc
- 670 pdc ont un champ 'nbre_pdc' mal documenté

In [89]:
itinerance_4.loc[~itinerance_4['nbre_pdc - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
478,9203,assistance-commerciale@e-vadea.fr,e-Vadea,"[2.969035, 42.9494317]",A9 - Aire de La Palme Ouest 11480 LA PALME (11),FREVAP11188A,e-Vadea - La Palme - A9 La Palme Ouest,Voirie,2,Accès libre,24/7,false,FREVAE11188AB2CHADEMO,2023-09-07,2023-11-23T06:37:21.950000+00:00
479,9204,assistance-commerciale@e-vadea.fr,e-Vadea,"[2.969035, 42.9494317]",A9 - Aire de La Palme Ouest 11480 LA PALME (11),FREVAP11188A,e-Vadea - La Palme - A9 La Palme Ouest,Voirie,2,Accès libre,24/7,false,FREVAE11188AB2T2,2023-09-07,2023-11-23T06:37:21.950000+00:00
480,9205,assistance-commerciale@e-vadea.fr,e-Vadea,"[2.969035, 42.9494317]",A9 - Aire de La Palme Ouest 11480 LA PALME (11),FREVAP11188A,e-Vadea - La Palme - A9 La Palme Ouest,Voirie,2,Accès libre,24/7,false,FREVAE11188AB3CCS1,2023-09-07,2023-11-23T06:37:21.950000+00:00
481,9206,assistance-commerciale@e-vadea.fr,e-Vadea,"[2.969035, 42.9494317]",A9 - Aire de La Palme Ouest 11480 LA PALME (11),FREVAP11188A,e-Vadea - La Palme - A9 La Palme Ouest,Voirie,2,Accès libre,24/7,false,FREVAE11188AB3CCS2,2023-09-07,2023-11-23T06:37:21.950000+00:00
482,9207,assistance-commerciale@e-vadea.fr,e-Vadea,"[2.969035, 42.9494317]",A9 - Aire de La Palme Ouest 11480 LA PALME (11),FREVAP11188A,e-Vadea - La Palme - A9 La Palme Ouest,Voirie,2,Accès libre,24/7,false,FREVAE11188AB3CHADEMO,2023-09-07,2023-11-23T06:37:21.950000+00:00
483,9208,assistance-commerciale@e-vadea.fr,e-Vadea,"[2.969035, 42.9494317]",A9 - Aire de La Palme Ouest 11480 LA PALME (11),FREVAP11188A,e-Vadea - La Palme - A9 La Palme Ouest,Voirie,2,Accès libre,24/7,false,FREVAE11188AB3T2,2023-09-07,2023-11-23T06:37:21.950000+00:00
484,9209,assistance-commerciale@e-vadea.fr,e-Vadea,"[2.969035, 42.9494317]",A9 - Aire de La Palme Ouest 11480 LA PALME (11),FREVAP11188A,e-Vadea - La Palme - A9 La Palme Ouest,Voirie,1,Accès libre,24/7,false,FREVAE11188AB4CCS,2023-09-07,2023-11-23T06:37:21.950000+00:00
485,9210,assistance-commerciale@e-vadea.fr,e-Vadea,"[2.969035, 42.9494317]",A9 - Aire de La Palme Ouest 11480 LA PALME (11),FREVAP11188A,e-Vadea - La Palme - A9 La Palme Ouest,Voirie,1,Accès libre,24/7,false,FREVAE11188AB4CHADEMO,2023-09-07,2023-11-23T06:37:21.950000+00:00
486,9211,assistance-commerciale@e-vadea.fr,e-Vadea,"[2.969035, 42.9494317]",A9 - Aire de La Palme Ouest 11480 LA PALME (11),FREVAP11188A,e-Vadea - La Palme - A9 La Palme Ouest,Voirie,1,Accès libre,24/7,false,FREVAE11188AB4T2,2023-09-07,2023-11-23T06:37:21.950000+00:00
487,9212,assistance-commerciale@e-vadea.fr,e-Vadea,"[2.969035, 42.9494317]",A9 - Aire de La Palme Ouest 11480 LA PALME (11),FREVAP11188A,e-Vadea - La Palme - A9 La Palme Ouest,Voirie,2,Accès libre,24/7,false,FREVAE11188AB5CCS1,2023-09-07,2023-11-23T06:37:21.950000+00:00


### coherence station - enseigne
- exemple de station avec plusieurs noms d'enseigne

In [96]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS27PFLEURYFORETCENTRE', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
34985,35687,pascal.lhermitte@siege27.fr,SIEGE59,"[1.556744, 49.419466]",Centre du village Fleury la Forêt,FRS27PFLEURYFORETCENTRE,900017,Voirie,2,Accès libre,24/7,false,FRS27EFLEURYFORETCENTRED,2021-06-15,2023-11-23T06:34:13.253000+00:00
34986,35688,pascal.lhermitte@siege27.fr,SIEGE60,"[1.556744, 49.419466]",Centre du village Fleury la Forêt,FRS27PFLEURYFORETCENTRE,900017,Voirie,2,Accès libre,24/7,false,FRS27EFLEURYFORETCENTREG,2021-06-15,2023-11-23T06:34:13.253000+00:00


In [97]:
itinerance_init.loc[itinerance_init.contact_operateur == 'pascal.lhermitte@siege27.fr', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
34886,35588,pascal.lhermitte@siege27.fr,SIEGE259,"[1.41371, 49.245858]",Place Jacques Ibert Les Andelys,FRS27PANDELYSNPOUSSIN,900117,Voirie,2,Accès libre,24/7,false,FRS27EANDELYSNPOUSSIND,2021-06-15,2023-11-23T06:34:13.253000+00:00
34887,35589,pascal.lhermitte@siege27.fr,SIEGE260,"[1.41371, 49.245858]",Place Jacques Ibert Les Andelys,FRS27PANDELYSNPOUSSIN,900117,Voirie,2,Accès libre,24/7,false,FRS27EANDELYSNPOUSSING,2021-06-15,2023-11-23T06:34:13.253000+00:00
34888,35590,pascal.lhermitte@siege27.fr,SIEGE261,"[1.401083, 49.23773]",Les Andelys Piscine,FRS27PANDELYSPTANDELYS,900118,Voirie,2,Accès libre,24/7,false,FRS27EANDELYSPTANDELYSD,2021-06-15,2023-11-23T06:34:13.253000+00:00
34889,35591,pascal.lhermitte@siege27.fr,SIEGE262,"[1.401083, 49.23773]",Les Andelys Piscine,FRS27PANDELYSPTANDELYS,900118,Voirie,2,Accès libre,24/7,false,FRS27EANDELYSPTANDELYSG,2021-06-15,2023-11-23T06:34:13.253000+00:00
34890,35592,pascal.lhermitte@siege27.fr,SIEGE187,"[1.352788, 49.174165]",Aubevoye Gare SNCF,FRS27PAUBEVOYEGARE,900082,Voirie,2,Accès libre,24/7,false,FRS27EAUBEVOYEGARED,2021-06-15,2023-11-23T06:34:13.253000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35144,35846,pascal.lhermitte@siege27.fr,SIEGE156,"[0.951704, 49.293602]",Rue de la Mairie Saint Ouen du Tilleul,FRS27PSTOUENTILLEULMAIRIE,900066,Voirie,2,Accès libre,24/7,false,FRS27PSTOUENTILLEULMAIRIEG,2021-06-15,2023-11-23T06:34:13.253000+00:00
35145,35847,pascal.lhermitte@siege27.fr,SIEGE217,"[1.22331, 49.231282]",Centre Bourg Saint Pierre de Vauvray,FRS27PSTPIERREVAUVRAYBOURG,900096,Voirie,2,Accès libre,24/7,false,FRS27PSTPIERREVAUVRAYBOURGD,2021-06-15,2023-11-23T06:34:13.253000+00:00
35146,35848,pascal.lhermitte@siege27.fr,SIEGE218,"[1.22331, 49.231282]",Centre Bourg Saint Pierre de Vauvray,FRS27PSTPIERREVAUVRAYBOURG,900096,Voirie,2,Accès libre,24/7,false,FRS27PSTPIERREVAUVRAYBOURGG,2021-06-15,2023-11-23T06:34:13.253000+00:00
35147,35849,pascal.lhermitte@siege27.fr,SIEGE163,"[0.975642, 49.260914]",ZAC Parking co voiturage Le Thuit Anger,FRS27PTHUITANGERCOVOITURE,900070,Voirie,2,Accès libre,24/7,false,FRS27PTHUITANGERCOVOITURED,2021-06-15,2023-11-23T06:34:13.253000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire

In [99]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues

In [100]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
50710,52156,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17161,2023-07-06,2023-11-23T06:34:35.324000+00:00
50711,52157,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17162,2023-07-06,2023-11-23T06:34:35.324000+00:00
50712,52158,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17163,2023-07-06,2023-11-23T06:34:35.324000+00:00
50713,52159,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17164,2023-07-06,2023-11-23T06:34:35.324000+00:00
50714,52160,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17165,2023-07-06,2023-11-23T06:34:35.324000+00:00
50715,52161,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17166,2023-07-06,2023-11-23T06:34:35.324000+00:00
50716,52162,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17167,2023-07-06,2023-11-23T06:34:35.324000+00:00
